In [1]:
import torch
import numpy as np
import pandas as pd
import sklearn
import re
import matplotlib.pyplot as plt

In [2]:
print(f"PyTorch version: {torch.__version__}")

# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")

# Set the device      
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

PyTorch version: 2.4.0
Is MPS (Metal Performance Shader) built? True
Is MPS available? True
Using device: mps


### Loading the textbook

In [3]:
with open("the-verdict.txt","r", encoding = "utf-8") as f:
    raw_text = f.read()
print(raw_text[:90])

I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so i


In [4]:
text = "hello , world. Is this -- a test?"
result = re.split(r'(\s)',text)
print(result)

['hello', ' ', ',', ' ', 'world.', ' ', 'Is', ' ', 'this', ' ', '--', ' ', 'a', ' ', 'test?']


In [5]:
text = "hello , world. Is this -- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)',text)
result = [item.strip() for item in result if item.strip()]
print(result)

['hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [6]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed), len(raw_text))

4690 20479


In [7]:
preprocessed[:100]

['I',
 'HAD',
 'always',
 'thought',
 'Jack',
 'Gisburn',
 'rather',
 'a',
 'cheap',
 'genius',
 '--',
 'though',
 'a',
 'good',
 'fellow',
 'enough',
 '--',
 'so',
 'it',
 'was',
 'no',
 'great',
 'surprise',
 'to',
 'me',
 'to',
 'hear',
 'that',
 ',',
 'in',
 'the',
 'height',
 'of',
 'his',
 'glory',
 ',',
 'he',
 'had',
 'dropped',
 'his',
 'painting',
 ',',
 'married',
 'a',
 'rich',
 'widow',
 ',',
 'and',
 'established',
 'himself',
 'in',
 'a',
 'villa',
 'on',
 'the',
 'Riviera',
 '.',
 '(',
 'Though',
 'I',
 'rather',
 'thought',
 'it',
 'would',
 'have',
 'been',
 'Rome',
 'or',
 'Florence',
 '.',
 ')',
 '"',
 'The',
 'height',
 'of',
 'his',
 'glory',
 '"',
 '--',
 'that',
 'was',
 'what',
 'the',
 'women',
 'called',
 'it',
 '.',
 'I',
 'can',
 'hear',
 'Mrs',
 '.',
 'Gideon',
 'Thwing',
 '--',
 'his',
 'last',
 'Chicago',
 'sitter',
 '--']

### Creating Vocabulary 

In [8]:
all_words = sorted(list(set(preprocessed)))
vocab_size = len(all_words)
print(vocab_size)

1130


In [9]:
all_words

['!',
 '"',
 "'",
 '(',
 ')',
 ',',
 '--',
 '.',
 ':',
 ';',
 '?',
 'A',
 'Ah',
 'Among',
 'And',
 'Are',
 'Arrt',
 'As',
 'At',
 'Be',
 'Begin',
 'Burlington',
 'But',
 'By',
 'Carlo',
 'Chicago',
 'Claude',
 'Come',
 'Croft',
 'Destroyed',
 'Devonshire',
 'Don',
 'Dubarry',
 'Emperors',
 'Florence',
 'For',
 'Gallery',
 'Gideon',
 'Gisburn',
 'Gisburns',
 'Grafton',
 'Greek',
 'Grindle',
 'Grindles',
 'HAD',
 'Had',
 'Hang',
 'Has',
 'He',
 'Her',
 'Hermia',
 'His',
 'How',
 'I',
 'If',
 'In',
 'It',
 'Jack',
 'Jove',
 'Just',
 'Lord',
 'Made',
 'Miss',
 'Money',
 'Monte',
 'Moon-dancers',
 'Mr',
 'Mrs',
 'My',
 'Never',
 'No',
 'Now',
 'Nutley',
 'Of',
 'Oh',
 'On',
 'Once',
 'Only',
 'Or',
 'Perhaps',
 'Poor',
 'Professional',
 'Renaissance',
 'Rickham',
 'Riviera',
 'Rome',
 'Russian',
 'Sevres',
 'She',
 'Stroud',
 'Strouds',
 'Suddenly',
 'That',
 'The',
 'Then',
 'There',
 'They',
 'This',
 'Those',
 'Though',
 'Thwing',
 'Thwings',
 'To',
 'Usually',
 'Venetian',
 'Victor',
 '

In [10]:
vocab = {token:integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i>50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)
('His', 51)


### Simple Tokenizer V1

In [13]:
vocab

{'!': 0,
 '"': 1,
 "'": 2,
 '(': 3,
 ')': 4,
 ',': 5,
 '--': 6,
 '.': 7,
 ':': 8,
 ';': 9,
 '?': 10,
 'A': 11,
 'Ah': 12,
 'Among': 13,
 'And': 14,
 'Are': 15,
 'Arrt': 16,
 'As': 17,
 'At': 18,
 'Be': 19,
 'Begin': 20,
 'Burlington': 21,
 'But': 22,
 'By': 23,
 'Carlo': 24,
 'Chicago': 25,
 'Claude': 26,
 'Come': 27,
 'Croft': 28,
 'Destroyed': 29,
 'Devonshire': 30,
 'Don': 31,
 'Dubarry': 32,
 'Emperors': 33,
 'Florence': 34,
 'For': 35,
 'Gallery': 36,
 'Gideon': 37,
 'Gisburn': 38,
 'Gisburns': 39,
 'Grafton': 40,
 'Greek': 41,
 'Grindle': 42,
 'Grindles': 43,
 'HAD': 44,
 'Had': 45,
 'Hang': 46,
 'Has': 47,
 'He': 48,
 'Her': 49,
 'Hermia': 50,
 'His': 51,
 'How': 52,
 'I': 53,
 'If': 54,
 'In': 55,
 'It': 56,
 'Jack': 57,
 'Jove': 58,
 'Just': 59,
 'Lord': 60,
 'Made': 61,
 'Miss': 62,
 'Money': 63,
 'Monte': 64,
 'Moon-dancers': 65,
 'Mr': 66,
 'Mrs': 67,
 'My': 68,
 'Never': 69,
 'No': 70,
 'Now': 71,
 'Nutley': 72,
 'Of': 73,
 'Oh': 74,
 'On': 75,
 'Once': 76,
 'Only': 77,
 '

In [11]:
# Creating a class
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [13]:
tokenizer = SimpleTokenizerV1(vocab)
text = "It's the last he painted, you know"
tokenizer.encode(text)

[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596]

### Simple Tokenizer V2

In [14]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>","<|unk|>"])
vocab = {token:integer for integer, token in enumerate(all_tokens)}
print(len(vocab.items()))

1132


In [21]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [22]:
text1 = "hello, do you like tea? We just got a cat"
text2 = "In the sunlit terraces of the palace"
text = " <|endoftext|> ".join((text1,text2))
print(text)

hello, do you like tea? We just got a cat <|endoftext|> In the sunlit terraces of the palace


In [23]:
tokenizer = SimpleTokenizerV2(vocab)
ids = tokenizer.encode(text)

In [24]:
ids

[1131,
 5,
 355,
 1126,
 628,
 975,
 10,
 107,
 590,
 503,
 115,
 1131,
 1130,
 55,
 988,
 956,
 984,
 722,
 988,
 1131]

In [25]:
tokenizer.decode(ids)

'<|unk|>, do you like tea? We just got a <|unk|> <|endoftext|> In the sunlit terraces of the <|unk|>'

### Byte Pair Encoding
This is how the GPT models were tokenised. We didnt use the <|unk|> keyword for the unknown words, but split them using the 
byte pair encoding. We use tiktoken to do this

In [26]:
from importlib.metadata import version
import tiktoken
print("tiktoken version:", version("tiktoken"))

tiktoken version: 0.7.0


In [27]:
tokenizer = tiktoken.get_encoding("gpt2")

In [28]:
text = "hello, do you like tea or protein powder? <|endoftext|> In the sunlit terrace lives a monster named Cristiano"
ids = tokenizer.encode(text, allowed_special = {"<|endoftext|>"})
print(ids)

[31373, 11, 466, 345, 588, 8887, 393, 7532, 11913, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 558, 3160, 257, 9234, 3706, 24568, 10115]


In [30]:
strings = tokenizer.decode(ids)
strings

'hello, do you like tea or protein powder? <|endoftext|> In the sunlit terrace lives a monster named Cristiano'

In [31]:
text_unk = "Akwirw doooer ier"
ids = tokenizer.encode(text_unk)
print(ids)

[33901, 86, 343, 86, 466, 2238, 263, 220, 959]


### Data Sampling using Sliding Window

In [33]:
with open("the-verdict.txt","r", encoding = "utf-8") as f:
    raw_text = f.read()

In [34]:
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [35]:
# removing first 50 as  the next is an interesting passage
enc_sample = enc_text[50:]
enc_sample

[290,
 4920,
 2241,
 287,
 257,
 4489,
 64,
 319,
 262,
 34686,
 41976,
 13,
 357,
 10915,
 314,
 2138,
 1807,
 340,
 561,
 423,
 587,
 10598,
 393,
 28537,
 2014,
 198,
 198,
 1,
 464,
 6001,
 286,
 465,
 13476,
 1,
 438,
 5562,
 373,
 644,
 262,
 1466,
 1444,
 340,
 13,
 314,
 460,
 3285,
 9074,
 13,
 46606,
 536,
 5469,
 438,
 14363,
 938,
 4842,
 1650,
 353,
 438,
 2934,
 489,
 3255,
 465,
 48422,
 540,
 450,
 67,
 3299,
 13,
 366,
 5189,
 1781,
 340,
 338,
 1016,
 284,
 3758,
 262,
 1988,
 286,
 616,
 4286,
 705,
 1014,
 510,
 26,
 475,
 314,
 836,
 470,
 892,
 286,
 326,
 11,
 1770,
 13,
 8759,
 2763,
 438,
 1169,
 2994,
 284,
 943,
 17034,
 318,
 477,
 314,
 892,
 286,
 526,
 383,
 1573,
 11,
 319,
 9074,
 13,
 536,
 5469,
 338,
 11914,
 11,
 33096,
 663,
 4808,
 3808,
 62,
 355,
 996,
 484,
 547,
 12548,
 287,
 281,
 13079,
 410,
 12523,
 286,
 22353,
 13,
 843,
 340,
 373,
 407,
 691,
 262,
 9074,
 13,
 536,
 48819,
 508,
 25722,
 276,
 13,
 11161,
 407,
 262,
 40123,
 18113,


In [36]:
print(tokenizer.decode(enc_sample[:20]))

 and established himself in a villa on the Riviera. (Though I rather thought it would have


In [37]:
context_size = 10
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y: {y}")

x: [290, 4920, 2241, 287, 257, 4489, 64, 319, 262, 34686]
y: [4920, 2241, 287, 257, 4489, 64, 319, 262, 34686, 41976]


In [38]:
for i in range(1, context_size+1):
    print(f"Value of i is {i}")
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, '------>', desired)

Value of i is 1
[290] ------> 4920
Value of i is 2
[290, 4920] ------> 2241
Value of i is 3
[290, 4920, 2241] ------> 287
Value of i is 4
[290, 4920, 2241, 287] ------> 257
Value of i is 5
[290, 4920, 2241, 287, 257] ------> 4489
Value of i is 6
[290, 4920, 2241, 287, 257, 4489] ------> 64
Value of i is 7
[290, 4920, 2241, 287, 257, 4489, 64] ------> 319
Value of i is 8
[290, 4920, 2241, 287, 257, 4489, 64, 319] ------> 262
Value of i is 9
[290, 4920, 2241, 287, 257, 4489, 64, 319, 262] ------> 34686
Value of i is 10
[290, 4920, 2241, 287, 257, 4489, 64, 319, 262, 34686] ------> 41976


In [39]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), '----------->', tokenizer.decode([desired]))

 and ----------->  established
 and established ----------->  himself
 and established himself ----------->  in
 and established himself in ----------->  a
 and established himself in a ----------->  vill
 and established himself in a vill -----------> a
 and established himself in a villa ----------->  on
 and established himself in a villa on ----------->  the
 and established himself in a villa on the ----------->  Riv
 and established himself in a villa on the Riv -----------> iera


### Creating dataset using Dataloader class

In [40]:
import torch
from torch.utils.data import Dataset , DataLoader

In [41]:
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids)-max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length +1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            
    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [42]:
def create_dataloader_v1(txt, batch_size = 4, max_length= 256, stride = 128, shuffle = True, drop_last = True):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(dataset, batch_size= batch_size, shuffle = shuffle, drop_last = drop_last)
    return dataloader

In [43]:
dataloader = create_dataloader_v1(raw_text, batch_size = 4, max_length = 4, stride = 4, shuffle = False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs", inputs)
print("targets", targets)

Inputs tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257]])
targets tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922]])


In [44]:
def get_nth_batch(dataloader, n):
    data_iter = iter(dataloader)
    for _ in range(n):
        batch = next(data_iter)
    return batch

In [45]:
n = 12
nth_batch = get_nth_batch(dataloader, n)
print(f"The {n}th batch input-output pair is:")
print(nth_batch)

The 12th batch input-output pair is:
[tensor([[  996,   484,   547, 12548],
        [  287,   281, 13079,   410],
        [12523,   286, 22353,    13],
        [  843,   340,   373,   407]]), tensor([[  484,   547, 12548,   287],
        [  281, 13079,   410, 12523],
        [  286, 22353,    13,   843],
        [  340,   373,   407,   691]])]


### Creating Token Embeddings

In [46]:
vocab_size = 5
output_dim = 4

In [47]:
torch.manual_seed(123)
embedding_layer= torch.nn.Embedding(vocab_size, output_dim)

In [48]:
embedding_layer

Embedding(5, 4)

This is basically making every word in the vocab into embeddings with output dim. So , the word at [0]th index can be represented by the numbers in the 3 Dimension space. Embedding layer is essentially a lookup ooperation that retrieves the row from the embedding layers

In [49]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.3035, -0.5880],
        [ 1.5810,  1.3010,  1.2753, -0.2010],
        [-0.1606, -0.4015,  0.6957, -1.8061],
        [-1.1589,  0.3255, -0.6315, -2.8400],
        [-0.7849, -1.4096, -0.4076,  0.7953]], requires_grad=True)


In [60]:
indices = torch.tensor([1,2,3,4,5,6,7,8])
embeddings = torch.nn.Embedding(len(indices), 4)
embeddings.weight

Parameter containing:
tensor([[-0.4921,  0.2942, -0.1224, -0.4060],
        [-0.6279, -0.0181,  1.1974,  0.0449],
        [ 1.5480,  0.9281, -0.6564,  1.7713],
        [-1.6067, -0.8521,  0.6658,  0.2453],
        [ 0.7206, -0.2908,  1.4274,  0.4548],
        [ 1.1140,  0.1213,  1.3954, -0.0235],
        [-0.5937,  0.0789, -0.2475,  1.3596],
        [-1.6196, -0.2544, -0.0432,  1.6211]], requires_grad=True)

In [65]:
embeddings.weight[0]

tensor([-0.4921,  0.2942, -0.1224, -0.4060], grad_fn=<SelectBackward0>)

### Encoding Word Positions

In [66]:
output_dim = 256
vocab_size = 50257
# The embedding layer will be of vocab_size x output_dim
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [68]:
(token_embedding_layer.weight)

Parameter containing:
tensor([[-0.4384, -0.6734, -0.5318,  ..., -1.6842,  0.0568, -1.7278],
        [-1.8346,  0.2576, -1.6789,  ...,  0.1348, -1.0571,  1.2703],
        [-0.5268,  1.3099,  0.5383,  ...,  0.0892,  0.2141, -1.4680],
        ...,
        [ 1.1078, -0.1039,  1.0237,  ...,  0.4691, -0.0714,  1.6100],
        [ 1.8363,  0.3762, -0.9555,  ...,  0.7549, -1.0260, -0.2873],
        [-0.2544, -0.3655,  0.8160,  ...,  0.0540, -0.6048, -0.2044]],
       requires_grad=True)

In [69]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [70]:
print("Tokens IDs:\n", inputs)
print("Target/Output IDs: \n", targets)

Tokens IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Target/Output IDs: 
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


Each token is embedded into a 256 dimension layer

In [79]:
token_embeddings = token_embedding_layer(inputs)
# We get a tensor of 8 matrices, each with 4 rows and 256 columns
# i.e 8 text samples with 4 Tokens each.
# Ex- [['we','are','going','there'],
#      ['but','who','are','you'],
#      ['is','their','a','football'],
#      ['yessir','player','best','Cristiano'],

print((token_embeddings.shape))

torch.Size([8, 4, 256])


In [74]:
token_embeddings

tensor([[[ 1.0673, -0.9578, -1.6891,  ..., -0.0593,  1.8501,  1.3731],
         [-0.6138, -0.2011, -0.0914,  ..., -0.3519, -0.1819,  0.8156],
         [-1.6583, -0.0482, -0.3877,  ..., -0.8098,  1.5039, -1.1519],
         [ 0.7303,  0.5204,  1.3259,  ...,  1.8956, -0.7623,  1.1655]],

        [[ 2.3072,  0.7848,  0.3885,  ...,  0.3978,  1.3345, -0.2019],
         [ 0.3628,  0.7024,  0.0107,  ..., -2.2250,  0.3137, -0.3477],
         [ 1.8633, -0.2696,  2.3642,  ...,  1.2405, -0.3353, -0.2682],
         [ 0.5673,  1.2943,  0.8292,  ...,  0.2302,  0.7557, -0.3102]],

        [[-0.9992, -0.1813, -0.0513,  ...,  1.8001,  0.0798,  0.7119],
         [ 0.8510,  1.4863, -0.4000,  ...,  0.1853,  0.8122, -0.7786],
         [ 1.3427,  1.1051,  0.6365,  ...,  0.1518,  1.8391,  0.6332],
         [-1.0850, -1.5062, -1.1121,  ..., -0.3098,  1.7297,  0.8462]],

        ...,

        [[-0.1118, -1.5130, -0.3220,  ..., -0.1557, -0.7694,  0.5798],
         [-0.3401, -2.2535,  0.9113,  ..., -0.0950, -0.36

GPT-2 uses absolute position embeddings, so we just create another embedding layer:

In [75]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [76]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [78]:
pos_embeddings

tensor([[ 0.2963,  1.1832,  1.5249,  ...,  1.3411, -0.5805,  0.4113],
        [ 1.6007,  2.9084,  1.3047,  ...,  0.4405, -0.2261, -0.2018],
        [ 0.8951, -0.1793,  1.0270,  ..., -1.5806, -0.9663,  0.4029],
        [ 0.6337, -0.5549, -1.3543,  ..., -1.2076, -1.6081,  1.2127]],
       grad_fn=<EmbeddingBackward0>)

In [77]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
